### 读入数据

In [ ]:
import re
str = "('吉林省长春市德惠市德兴街与惠新路交汇西行100米德惠市政务服务中心二楼社保窗口',)"

cleaned_text = re.sub(r"^\(|\)$|'|,$", "", str)
print(cleaned_text)

In [ ]:
import pandas as pd

# 读取两个 Excel 文件
df1 = pd.read_excel("./全市.xlsx")
df2 = pd.read_excel("./新区.xlsx")



In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
# 按行合并（忽略原有索引）
merged_df = pd.concat([df1, df2], ignore_index=True)

# 保存结果
merged_df.to_excel("./全市全部.xlsx", index=False)

merged_df.info()

In [ ]:
# 读取两个 Excel 文件
import pandas as pd
df= pd.read_excel("./全市全部.xlsx")
df.info()

In [ ]:
# 2. 分组统计不重复区划名称数量
import pandas as pd
df = pd.read_excel("./excel_main0425.xlsx")

result = df.groupby(['主项名称', '业务办理项名称'])['区划名称'].nunique().reset_index()
result.rename(columns={'区划名称': '不重复区划数'}, inplace=True)

# 导出到Excel
output_file = "按区划统计主项.xlsx"

result.to_excel(output_file, index=False)

In [ ]:
# 2. 分组统计不重复区划数
grouped_stats = df.groupby(['主项名称', '业务办理项名称'])['区划名称'].agg([
    ('不重复区划数', 'nunique'),
    ('区划列表', lambda x: list(x.unique()))
]).reset_index()

# 3. 计算占比
total_groups = len(grouped_stats)  # 总分组数
groups_gt_5 = len(grouped_stats[grouped_stats['不重复区划数'] > 20])  # 不重复区划数>5的分组数
percentage = (groups_gt_5 / total_groups) * 100  # 占比

# 4. 输出结果
print("==== 分组统计详情 ====")
print(grouped_stats.to_string(index=False))

print("\n==== 占比计算结果 ====")
print(f"总分组数: {total_groups}")
print(f"不重复区划数 > 20 的分组数: {groups_gt_5}")
print(f"占比: {percentage:.2f}%")

In [11]:
# import pandas as pd

# # 1. 读取数据（假设数据为CSV格式）
# df = pd.read_csv('data.csv')  # 替换为你的文件路径

# 2. 分组统计不重复区划数
grouped_stats = df.groupby(['主项名称', '业务办理项名称'])['区划名称'].nunique().reset_index(name='不重复区划数')

# 2. 计算关键分位数
quantiles = grouped_stats['不重复区划数'].quantile([0.25, 0.5, 0.75, 0.9]).to_dict()

# 3. 输出分布区间
print("==== 不重复区划数分布区间 ====")
print(f"25%分位数: {quantiles[0.25]:.1f} (25%的分组 ≤ 此值)")
print(f"中位数: {quantiles[0.5]:.1f}")
print(f"75%分位数: {quantiles[0.75]:.1f} (75%的分组 ≤ 此值)")
print(f"90%分位数: {quantiles[0.9]:.1f} (90%的分组 ≤ 此值)")

# 4. 判断主要分布区间
iqr = quantiles[0.75] - quantiles[0.25]  # 四分位距
main_range = (quantiles[0.25], quantiles[0.75])
print(f"\n核心分布区间（IQR）: {main_range[0]:.1f} ~ {main_range[1]:.1f}")
print(f"解释: 约50%的数据集中在此范围内")


==== 不重复区划数分布区间 ====
25%分位数: 7.0 (25%的分组 ≤ 此值)
中位数: 13.0
75%分位数: 16.0 (75%的分组 ≤ 此值)
90%分位数: 17.0 (90%的分组 ≤ 此值)

核心分布区间（IQR）: 7.0 ~ 16.0
解释: 约50%的数据集中在此范围内


### 按主项名称对业务办理项名称的数量进行统计

In [ ]:
# 按'主项名称'分组，并对'业务办理项名称'去重计数
result = df.groupby('主项名称')['业务办理项名称'].nunique().reset_index()
result.columns = ['主项名称', '业务办理项数量(去重)']

# 导出到Excel
output_file="按数量统计主项.xlsx"

result.to_excel(output_file, index=False)

### 按主项名称对业务办理项名称的数量进行统计，带上部门名称

In [ ]:
# 检查必要的列是否存在
# 智能客服4.18全市基本信息
import pandas as pd
df = pd.read_excel("./智能客服4.18全市基本信息.xlsx")
required_columns = ['主项名称', '业务办理项名称', '部门名称']  # 添加了'部门'列
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"数据中缺少必要的列: '{col}'")

# 按'主项名称'分组，并对'业务办理项名称'去重计数，同时保留部门信息
# 方法1：假设每个主项名称对应唯一的部门
result = df.groupby('主项名称').agg({
    '业务办理项名称': 'nunique',
    '部门名称': 'first'  # 取每个组中的第一个部门值
}).reset_index()

# 重命名列
result.columns = ['主项名称', '业务办理项数量(去重)', '部门名称']

# 调整列顺序，让部门信息在前
result = result[['主项名称', '部门名称', '业务办理项数量(去重)']]
# 导出到Excel
output_file = "按数量统计主项带部门.xlsx"
# 导出到Excel
result.to_excel(output_file, index=False)
print(f"结果已成功导出到: {output_file}")

In [ ]:
# 查找主项名称包含"公积金"的不重复子项名称
result = df[df['主项名称'].str.contains('公积金')]['子项名称'].drop_duplicates()

result.head()

In [ ]:
# 查找主项名称包含"公积金"的记录，并按子项名称去重
result = df[df['主项名称'].str.contains('公积金')].drop_duplicates(subset=['子项名称'])

print(result[['主项名称', '子项名称']])
export = result[['主项名称', '子项名称']]
export.to_excel("子项名称(公积金).xlsx", index=False)

### 导出包含 企业登记注册 的数据

In [ ]:
# 查找主项名称包含"公积金"的记录，并按子项名称去重
result = df[df['主项名称'].str.contains('企业登记注册')].drop_duplicates(subset=['业务办理项名称'])

# print(result[['主项名称', '子项名称']])
export = result[['主项名称', '业务办理项名称']]
export.to_excel("企业登记注册.xlsx", index=False)

### 导出不重复主项-业务办理项-情形

In [ ]:
# df = pd.read_excel("./全市全部.xlsx")
data = df[['主项名称', '业务办理项名称']].drop_duplicates()
data = data[['主项名称', '业务办理项名称']]
data.to_excel("不重复主项-业务办理项.xlsx", index=False)

In [ ]:
# df = pd.read_excel("./全市全部.xlsx")
data =  df[['主项名称', '业务办理项名称', '情形']].drop_duplicates()
data = data[['主项名称', '业务办理项名称', '情形']]
data.to_excel("不重复主项-业务办理项-情形.xlsx", index=False)

In [ ]:
df3 = pd.read_excel("./智能客服4.18全市基本信息.xlsx")
data =  df3[['主项名称',"部门名称", '业务办理项名称', '情形']].drop_duplicates()
data = data[["部门名称", '主项名称', '业务办理项名称', '情形']]
data.to_excel("不重复部门-主项-业务办理项-情形.xlsx", index=False)